In [33]:
import time
import numpy as np
import pandas as pd
import random
#pd.options.display.max_columns = None
#pd.options.display.mpl_style = 'default'

import re
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

from scipy import sparse

In [34]:
training_ratio  = 1

n = round(len(np.load('preprocessed_data/train_title_char.npy'))* training_ratio)

In [35]:
train_title_char = np.load('preprocessed_data/train_title_char.npy')[:n]
train_title_word = np.load('preprocessed_data/train_title_word.npy')[:n]
test_title_char = np.load('preprocessed_data/test_title_char.npy')
test_title_word = np.load('preprocessed_data/test_title_word.npy')

train_labels = np.load('preprocessed_data/labels.npy').item()[:n]
label_dictionary = np.load('preprocessed_data/label_dictionary.npy').item()

In [36]:
char_embedding = np.load('preprocessed_data/char_embedding.npy')
word_embedding = np.load('preprocessed_data/word_embedding.npy')

In [37]:
train_len = np.load('preprocessed_data/train_length.npy')
test_len = np.load('preprocessed_data/test_length.npy')

train_title_char_len = train_len[0][:n]
train_title_word_len = train_len[1][:n]
test_title_char_len = test_len[0]
test_title_word_len = test_len[1]

## Define training and validation set

In [40]:
title = 'char'
slicing_index = -20000
train_size = train_title_char.shape[0]

if title  == 'char':
    train_title = train_title_char[:slicing_index]
    train_title_length = train_title_char_len[:slicing_index]
    train_label = train_labels[:slicing_index]
    val_title = train_title_char[slicing_index:]
    val_title_length = train_title_char_len[slicing_index:]
    val_label = train_labels[slicing_index:]
    test_title = test_title_char
    test_title_length = test_title_char_len    
    embedding = char_embedding
elif title  == 'word':
    train_title = train_title_word[:slicing_index]
    train_title_length = train_title_word_len[:slicing_index]
    train_label = train_labels[:slicing_index]
    val_title = train_title_word[slicing_index:]
    val_title_length = train_title_word_len[slicing_index:]
    val_label = train_labels[slicing_index:]
    test_title = test_title_word
    test_title_length = test_title_word_len    
    embedding = word_embedding

## Title Charactor RNN

In [41]:
def evalation(predict_label_and_marked_label_list):

    right_label_num = 0  
    right_label_at_pos_num = [0, 0, 0, 0, 0] 
    sample_num = 0   
    all_marked_label_num = 0    
    for predict_labels, marked_labels in predict_label_and_marked_label_list:
        sample_num += 1
        marked_label_set = set(marked_labels)
        all_marked_label_num += len(marked_label_set)
        for pos, label in zip(range(0, min(len(predict_labels), 5)), predict_labels):
            if label in marked_label_set:     
                right_label_num += 1
                right_label_at_pos_num[pos] += 1

    precision = 0.0
    for pos, right_num in zip(range(0, 5), right_label_at_pos_num):
        precision += ((right_num / float(sample_num))) / np.log(2.0 + pos)  
    recall = float(right_label_num) / all_marked_label_num

    return (precision * recall) / (precision + recall)

def label_index_extraction(x):
    lst =[]
    for i, j in enumerate(x):
        if j == 1:
            lst.append(i)
    return lst

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [42]:
vocab_size = embedding.shape[0]
RNN_hidden_size = 256
target_size = train_label.shape[1]

In [43]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
sess = tf.InteractiveSession()

title = tf.placeholder(tf.int64, [None, None], "title")                # [batch_size x max_len]
title_length = tf.placeholder(tf.int64, [None], "title_length")        # [batch_size]
labels = tf.placeholder(tf.int64, [None, target_size], "label")        # [batch_size x target_size]

initializer = tf.random_uniform_initializer(-0.5, 0.5)
embeddings = tf.get_variable("embeddings", [vocab_size, 256], initializer=initializer, trainable= True)
embeddings = embeddings.assign(embedding) 

title_embedded = tf.nn.embedding_lookup(embeddings, title)  

### ------------------------------------------------------------------------------------------------------- ### 
### ---------------------------------------- sentence encoders -------------------------------------------  ###
### ------------------------------------------------------------------------------------------------------- ### 

lstm_title_cell = tf.contrib.rnn.LSTMCell(RNN_hidden_size, state_is_tuple= True)
#lstm_title_cell = tf.contrib.rnn.GRUCell(RNN_hidden_size)
_, title_final_state = tf.nn.dynamic_rnn(lstm_title_cell, title_embedded, sequence_length = title_length, dtype=tf.float32)        
    
title_vector = title_final_state.h  
title_vector = tf.layers.batch_normalization(title_vector)
### ------------------------------------------------------------------------------------------------------- ### 
### ---------------------------------------- loss & prediction -------------------------------------------  ###
### ------------------------------------------------------------------------------------------------------- ###     

h = tf.contrib.layers.linear(title_vector, 256, activation_fn = tf.nn.relu)
h = tf.layers.batch_normalization(h) 
logits = tf.contrib.layers.linear(h, target_size)

probability = tf.nn.sigmoid(logits)
loss = tf.losses.sigmoid_cross_entropy(logits = logits, multi_class_labels = labels)
                                                                                    
opt_op = tf.train.AdamOptimizer().minimize(loss)
saver = tf.train.Saver()

print('graph created')

graph created


In [49]:
BATCH_SIZE = 200
with tf.Session() as sess:  
    #sess.run(tf.global_variables_initializer())
    saver.restore(sess, "saved_models/lstm_baseline_model_6.ckpt")
    n = train_title.shape[0]
    
    part_train_title = train_title[:1000]
    part_train_length = train_title_length[:1000]
    train_feed_dict = {title: part_train_title, title_length: part_train_length}
    
    train_label_index = [label_index_extraction(i) for i in list(train_label[:1000].toarray())] 
    val_label_index = [label_index_extraction(i) for i in list(val_label.toarray())] 
    val_title_split = list(split(val_title, 20))
    val_title_length_split = list(split(val_title_length, 20))
    val_predicted = []
    
    for epoch in range(15):
        print('----- Epoch', epoch + 1, '-----')
        total_loss = 0
        t_0 = time.time()
        
        for i in range(n// (BATCH_SIZE )):
            index_list = random.sample(range(n), BATCH_SIZE )
            inst_train_title = [train_title[idx] for idx in index_list]
            inst_train_length = [train_title_length[idx] for idx in index_list]
            inst_train_labels = train_label[index_list].toarray()
            feed_dict = {title: inst_train_title, title_length: inst_train_length, labels: inst_train_labels}

            _, current_loss = sess.run([opt_op, loss], feed_dict=feed_dict)
            total_loss = current_loss + total_loss
            
        print(' Train loss:', total_loss, 'Time:', round((time.time()-t_0)/60,2),'minute') 
        #save_path = saver.save(sess, "saved_models/lstm_baseline_model_%s.ckpt" % (epoch + 1))
              
        train_probability = sess.run([probability], feed_dict=train_feed_dict)
        train_predicted_labels = [list(np.argsort(i)[::-1][:5]) for i in train_probability[0]]
        train_label_pairs = list(zip(train_predicted_labels, train_label_index))
        train_f1 = evalation(train_label_pairs)
        print('train (partial) f1: ', train_f1)
        
        for i in range(20):
            val_feed_dict = {title: val_title_split[i], title_length: val_title_length_split[i]}
            val_probability = sess.run([probability], feed_dict=val_feed_dict)
            val_predicted_labels = [list(np.argsort(i)[::-1][:5]) for i in val_probability[0]]
            val_predicted = val_predicted + val_predicted_labels 
        val_label_pairs = list(zip(val_predicted, val_label_index))
        val_f1 = evalation(val_label_pairs)
        print('validaton f1: ', val_f1)

INFO:tensorflow:Restoring parameters from saved_models/lstm_baseline_model_6.ckpt
----- Epoch 1 -----
 Train loss: 63.3156142624 Time: 47.23 minute
train (partial) f1:  0.435999375943
validaton f1:  0.36964792703
----- Epoch 2 -----
 Train loss: 62.9856615735 Time: 50.08 minute
train (partial) f1:  0.438510953905
validaton f1:  0.36964792703
----- Epoch 3 -----


KeyboardInterrupt: 

## Eval Prediction